In [ ]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings # CAMBIO: Librería más estable
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from pinecone import Pinecone, ServerlessSpec
import os
import time # NUEVO: Para darle tiempo a Pinecone de "despertar"
from dotenv import load_dotenv

load_dotenv()

# 1. Cargar el Dataset
df = pd.read_csv(r"C:\Users\MSI\proyecto-multivariado\dataset.csv")

# 2. Preprocesamiento
docs = []
for i, row in df.iterrows():
    doc = Document(
        page_content=str(row['abstract']),
        metadata={
            "title": str(row['title']),
            "authors": str(row['authors']),
            "venue": str(row['venue']),
            "year": int(row['year'])
        }
    )
    docs.append(doc)

# 3. GENERAR EMBEDDINGS GRATUITOS
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Configurar Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "academic-search"

# Lógica de borrado y creación con pausas
if index_name in pc.list_indexes().names():
    print("Borrando índice antiguo para actualizar dimensiones...")
    pc.delete_index(index_name)
    time.sleep(10) # Espera a que el borrado se procese en la nube

print("Creando nuevo índice de 384 dimensiones...")
pc.create_index(
    name=index_name,
    dimension=384, 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

# --- CLAVE PARA EVITAR EL ERROR 503 ---
print("Esperando 20 segundos para que los servidores de Pinecone estén listos...")
time.sleep(20) 
# --------------------------------------

# Guardar en Pinecone
vectorstore = PineconeVectorStore.from_documents(
    docs, 
    embeddings, 
    index_name=index_name
)

print(f"¡Listo! Se han cargado {len(docs)} documentos a Pinecone exitosamente.")